# Conception de réseaux de neurones convolutifs pour le diagnostic de pneumonies
*Théo Gachet - Septembre 2021*

### Contexte et objectifs : un projet mêlant Deep Learning et imagerie médicale

Dans le cadre de mon TIPE (Travaux d'Initiative Personnelle "Encadrés") en classe préparatoire scientifique MP* au Lycée Saint-Louis (Paris), j'ai utilisé des techniques d’apprentissage automatique pour obtenir un algorithme effectuant le diagnostic de pneumonies à partir de radiologies. Ce projet, mêlant imagerie médicale et réseaux de neurones convolutifs, répond au thème national de l'année 2021-2022 et commun à tous les étudiants de CGPE : *Santé et Prévention*.

### Librairies et modules

Commençons par importer toutes les librairies, tous les modules ainsi que les paramètres nécessaires au projet

- Scikit     : pip install -U scikit-learn
- Numpy      : pip install numpy
- OpenCV     : pip install opencv-python
- Pandas     : pip install pandas
- MatPlotLib : pip install matplotlib
- Tensorflow : https://www.tensorflow.org/install

On importe des éléments de base qui permettront de gérer les données et de tracer des graphes

In [ ]:
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import pandas as pd 
import numpy as np
import random
import cv2
import os

from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, SeparableConv2D, MaxPool2D, LeakyReLU, Activation
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

input_path = 'F:/TIPE/chest_xray/'

### Nombres pseudo-aléatoires

Le problème des ordinateurs est qu'ils sont déterminés, non pas aléatoires. Ainsi, les générateur de nombre pseudo-aléatoires permettent de produire des nombre qui semblent aléatoires mais sont en fait déterminés car ils dépendant des paramètres d'entrée. La fonction np.random.seed(seed) génère ainsi l'entrée (le seed) à l'algorithme qui génère des nombre pseudo aléatoires avec NumPy. 

On veut utiliser des nombre pseudo-aléatoires car ils permettent de rendre le code ré-exécutable de la même manière. Le nombre généré a un aspect aléatoire pour que l'on voit si le code fonctionne bien mais on peut aussi vouloir garder les mêmes valeurs d'entrée pour avoir des analyses de sortie cohérentes et rigoureuses.

In [ ]:
seed = 232
np.random.seed(seed)
tf.random.set_seed(seed)

### Echantillonnage des données

Ensuite, j'ai séparé les ensembles de données en trois sous-ensembles :
- train : les données d'entraînement
- val   : les données de validation
- test  : les données de test

Puis j'ai affiché le contenu de ces différents ensembles afin de savoir la quantité de données sur laquelle nous allions travailler

In [ ]:
for _set in ['train', 'val', 'test']:
    n_normal = len(os.listdir(input_path + _set + '/NORMAL'))
    n_infect = len(os.listdir(input_path + _set + '/PNEUMONIA'))
    print('Ensemble : {} / Scans sains : {} / Scans de pneumonie : {}'.format(_set, n_normal, n_infect))

### Prétraitement des images (preprocessing)

Pour la phase de preprocessing, j'ai défini une fonction qui permet de traiter les données et de charger l'ensemble des valeurs d'entraînement et de test dans le réseau. J'ai aussi créé des labels pour les images.

On s'attaque ici à de la reconnaissance d'image mais le jeu d’apprentissage est très petit : c'est embêtant quand on sait que les algorithmes d’aujourd’hui raffolent de gros volumes d’images labellisées !

Une des méthodes les plus utilisées pour répondre à ce problème est la Data Augmentation (DA).

### Data Augmentation

En analyse de données, cette méthode est utilisée pour augmenter la quantité de données en ajoutant des copies très légèrement modifiées à ceux qui existent déjà, ou bien en créant de nouvelles données à partir d'une consigne précise (on utilisera la première méthode ici).

Rq : ainsi, dans le code, "datagen" signifie "data generation objects".

Cela permet de régulariser les données et participe à réduire le surapprentissage (overfitting) lorsqu'on entraîne un modèle de machine learning. Cette méthode est parfois comparée à l'oversampling en musique (cela consiste à construire un morceau en enregistrant à l'aide d'un sampler plusieurs bribes successives les unes par-dessus les autres).

Dans notre projet, la Data Augmentation modifie la taille du dataset (du training set, pour être précis) en modifiant l'aspect des images, sans modifier leur sémantique (leur essence). Une fois que le training set est plus diversifié, le réseau va être capable, pendant sa phase d'entraînement, de prendre plus de recul sur les données.

Dans le code, on a :
- rescale=1./255     : on redimentionne les images
- zoom_range=0.3     : on effectue un zoom de ration 0.3
- vertical_flip=True : on applique une rotation

### Générateurs de données

On définit ensuite deux générateurs de données :
- train_gen pour les données d'entraînement
- test_gen pour les données de validation

Ces générateurs permettent de charger les données nécessaires directement depuis le dossier source et les convertissent en données d'entraînement qui seront injectées dans le modèle.

In [ ]:
def process_data(img_dims, batch_size):

    train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, vertical_flip=True)
    test_val_datagen = ImageDataGenerator(rescale=1./255)
    
    train_gen = train_datagen.flow_from_directory(
        directory=input_path+'train', 
        target_size=(img_dims, img_dims), 
        batch_size=batch_size, 
        class_mode='binary', 
        shuffle=True)

    test_gen = test_val_datagen.flow_from_directory(
        directory=input_path+'test', 
        target_size=(img_dims, img_dims), 
        batch_size=batch_size, 
        class_mode='binary', 
        shuffle=True)
    
    test_data = []
    test_labels = []

    for cond in ['/NORMAL/', '/PNEUMONIA/']:
        for img in (os.listdir(input_path + 'test' + cond)):
            img = plt.imread(input_path+'test'+cond+img)
            img = cv2.resize(img, (img_dims, img_dims))
            img = np.dstack([img, img, img])
            img = img.astype('float32') / 255
            if cond=='/NORMAL/':
                label = 0
            elif cond=='/PNEUMONIA/':
                label = 1
            test_data.append(img)
            test_labels.append(label)
        
    test_data = np.array(test_data)
    test_labels = np.array(test_labels)
    
    return train_gen, test_gen, test_data, test_labels

### Hyperparamètres du réseau

Dans cette partie, j'ai juste défini quelques constantes qui seront utiles pour la suite :

- img_dims = 150   : dimension des images (pour la phase de preprocessing)

- epochs = 10      : l'epoch ("époque" ou "ère" en français) indique le nombre de passages dans l'ensemble des données de test que va parcourir l'algorithme de machine learning. En général, les datasets sont groupés en batches (surtout quand le nombre de données est élevé). On peut utiliser le terme d'itération pour parler du fait d'injecter un batch dans le modèle (cela représente donc 1 itération).

- batch_size = 32  : la taille du batch prendra en général une valeur entre 32 et 128, on peut faire varier cette valeur en fonction de la puissance de calcul de l'ordinateur et du niveau de précision que l'on attend du modèle. 

*batch_size* est le nombre d'exemples qui sont "montrés" à l'algorithme avant que les poids/paramètres du modèle soient recalculés (le gradiant est calculé pour chaque sous-ensemble de cardinal batch_size et non sur toutes les données à la fois afin de contourner les problèmes de mémoire et de traitement). Cela revient à ajuster le modèle tous les n exemples, où n = batch_size, au lieu de l'ajuster après chacun de ces exemples. Cela a tendance a faciliter l'apprentissage et éviter certaines dérives qui peuvent compliquer l'apprentissage.

Par exemple, si la taille du batch est égale à l'entièreté de l'ensemble d'entraînement, alors le nombre d'epochs est le nombre d'itérations.

La relation générale est : d \* e = i \* b, avec :
- d = taille du dataset
- e = nombre d'epochs
- i = nombre d'itérations
- b = taille du batch


In [ ]:
img_dims = 150
epochs = 5
batch_size = 32

### Données de t'entraînement et de test

On applique la fonction de preprocessing définie précédemment à nos générateurs de données d'entrainement, de test, à nos données de test et à nos labels (les labels sont des listes de 0=normal ou de 1=pneumonie)

In [ ]:
train_gen, test_gen, test_data, test_labels = process_data(img_dims, batch_size)     # 9

### Architecture du réseau de neurones convolutifs 

Vient ensuite la partie cruciale de construction du modèle du réseau de neurones convolutifs.

Une architecture de réseau de neurones convolutifs est formée par un empilement de couches de traitement :
- la couche de convolution (CONV) qui traite les données d'un champ récepteur
- la couche de pooling (POOL), qui permet de compresser l'information en réduisant la taille de l'image intermédiaire (souvent par sous-échantillonnage)
- la couche de correction (ReLU), souvent appelée par abus « ReLU » en référence à la fonction d'activation (Unité de rectification linéaire)
- la couche « entièrement connectée » (FC), qui est une couche de type perceptron
- la couche de perte (LOSS)

Pour ce projet, j'ai assemblé 5 blocs convolutifs composés d'une couche de convolution, d'une couche de pooling) et d'une couche de batch-normalization.

### Couche de convolution et hyperparamètres

La couche de convolution est le bloc de construction de base d'un réseau de neurones convolutifs. Pour dimensionner le volume de sortie (aussi appelé "volume de la couche de convolution"), on définit 3 hyperparamètres : la profondeur, le pas et la marge (en apprentissage automatique, un hyperparamètre est un paramètre dont la valeur est utilisée pour contrôler le processus d'apprentissage).

- La profondeur de la couche correspond au nombre de neurones associés à un même champ recepteur (*) (nombre de noyaux de convolution).

- Le pas permet de préciser le chevauchement des champs récepteurs. Plus il est faible, plus les champs recepteurs se chevauchent et plus le volume de sortie sera grand.

- La taille de la marge (à 0), aussi appelée "zero padding". Cette marge consiste à mettre des zéros à la frontière du volume d'entrée, elle permet de contrôler la dimension spatiale du volume de sortie, ie le nombre de champs récepteurs à gérer. En effet, il est parfois souhaitable de conserver la même surface que celle du volume d'entrée.

(*) Champ recepteur d'un neurone : volume de l'espace qui modifie la réponse de ce neurone, quand un stimulus suffisamment puissant et rapide survient en son sein. De tels champs récepteurs ont été identifiés dans les systèmes visuel, auditif et somatosensoriel. Ainsi, le champ récepteur d'un neurone du système visuel est la portion du champ visuel qui, lorsqu'on présente un stimulus lumineux en son sein, modifie la réponse de ce neurone. Cette définition a été étendue à des espaces plus abstraits qui décrivent les paramètres possibles d'une stimulation. Par exemple, certains neurones du système visuel ne sont excités que par certaines longueurs d'onde. On peut donc définir le champ récepteur d'un neurone comme le sous-ensemble des paramètres des stimulations qui modifient son activité.

### Couche de pooling

Le pooling ("mise en commun" en français) est une forme de sous-échantillonnage de l'image. On divise l'image d'entrée en une série de cellules de n pixels de côté ne se chevauchant pas. Le signal en sortie de cellule est défini en fonction des valeurs prises par les différents pixels de la cellule. Le pooling diminue la taille spatiale d'une image intermédiaire, ce qui permer de diminuer la quantité de paramètres et le nombre de calculs dans le réseau. Elle peut donc réduire le sur-apprentissage et crée aussi une forme d'invariance par translation.

La forme la plus courante est une couche de mise en commun avec des cellules de taille 2 × 2 et comme valeur de sortie la valeur maximale en entrée. On parle dans ce cas de « Max-Pool 2x2 » (compression d'un facteur 4).

### Couche de batch normalization : 

La "batch normalization" est une méthode permettant de rendre un réseau de neurones artificiels plus rapide et plus stable grâce à la normalisation des couches d'entrées en recentrant et en redimentionnant les données reçues.

Aujourd'hui encore, son efficacité reste empirique et le fonctionnement théorique de la méthode n'est pas complètement maîtrisée. L'explication la plus commune est que cette méthode réduit le "décalage de covariable" (ou déplacement covarié).

Le décalage de covariable est la variation de la distribution d'un réseau d'éctivation à cause des paramètres d'entraînement.

Dans un réseau de neurones, la sortie de la première couche est envoyée à l'entrée de la deuxième etc,. Quand les paramètres d'une couche changent, alors la distribution des entrées des couches suivantes est aussi modifiée.

Ces variations dans la distribution des entrées peut devenir problématique pour le réseau de neurones, en particulier lorsqu'on a un grand nombre de couches.

### Couche de correction

Pour améliorer les performances du réseau de neurones, on fait appel à une couche de correction qui va opérer une fonction mathématique (dite fonction d'activation) sur la sortie des couches. Plusieurs fonctions existent :

- La correction par tangente hyperbolique :                 f(x)= tanh(x)
- La correction par la tangente hyperbolique saturante :    f(x)= |tanh(x)|
- La correction par la fonction sigmoïde :                  f(x)= (1+e^(-x))^(-1)
- La correction ReLU (Unité Linéaire Rectifiée) :           f(x)= max(0,x)

Souvent, la correction Relu est préférable, car il en résulte la formation de réseau neuronal plus rapide, sans faire une différence significative à la généralisation de précision (*). Cette fonction, appelée aussi « fonction d'activation non saturante », augmente les propriétés non linéaires de la fonction de décision et de l'ensemble du réseau sans affecter les champs récepteurs de la couche de convolution.

(\*) Relu augmente la rapidité du réseau : *A. Krizhevsky, I. Sutskever et G. E. Hinton, « ImageNet Classification with Deep Convolutional Neural Networks », Advances in neural Processing Systems de traitement, vol. 1, 2012, p. 1097–1105 (16 février 2015)*


In [ ]:
inputs = Input(shape=(img_dims, img_dims, 3))

# 1er bloc convolutif
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# 2ème bloc convolutif
x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# 3ème bloc convolutif
x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# 4ème bloc convolutif
x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)
# 5ème bloc convolutif
x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)

### Couche entièrement connectée
Après avoir construit les 5 blocs convolutifs, j'ai appliqué une couche dite "entièrement connectée". Ces couches connectent chaque neurone d'une couche à chaque neurone d'une autre couche, elle constitue toujours la dernière couche d'un réseau de neurones.

En entrée, cette couche reçoit un vecteur et produit un nouveau vecteur en sortie qui est obtenu en appliquant une combinaison linéaire (et éventuellement une fonction d'activation) aux valeurs du vecteur d'entrée.

Cette couche renvoie un vecteur de taille N, où N est le nombre de classes dans notre problème de classification d'images. Chaque élément du vecteur indique la probabilité pour l'image en entrée d'appartenir à une classe. Par exemple, si le premier élément représente la probabilité d'avoir un scan "normal" et que le second représente la probabilité d'avoir un scan "pneumonie", le vecteur final [0.3 ; 0.7] signifie que l'image a 70% de chances d'appartenir à la classe "pneumonie".

Chaque valeur du vecteur d'entrée "vote" en faveur d'une classe. Les votes n'ont pas tous la même importance car la couche leur accorde des poids qui dépendent de l'élément du tableau et de la classe.

Pour calculer les probabilités, la couche entièrement connectée multiplie chaque élément en entrée par un poids, fait la somme, puis applique une fonction d'activation. Ce traitement revient à multiplier le vecteur en entrée par la matrice contenant les poids. Le fait que chaque valeur en entrée soit connectée avec toutes les valeurs en sortie explique le terme "entièrement connecté".

Comment connait-on la valeur de ces poids ? Le réseau de neurones convolutif apprend les valeurs des poids de la même manière qu'il apprend les filtres de la couche de convolution : lors de phase d'entraînement, par rétropropagation du gradient.

La couche entièrement connectée détermine le lien entre la position des features dans l'image et une classe. En effet, le vecteur en entrée est issu de la couche précédente, il correspond à une carte d'activation pour une feature donnée : les valeurs élevées indiquent la localisation (plus ou moins précise selon le pooling) de cette feature dans l'image. Si la localisation d'une feature à un certain endroit de l'image est caractéristique d'une certaine classe, alors on accorde un poids important à la valeur correspondante dans le vecteur.

### Méthode du décrochage/dropout

Dans chaque bloc, j'ai appliqué la méthode du décrochage ("dropout"). C'est une technique de régularisation pour réduire l'overfitting (surajustement) dans les réseaux de neurones. Cela permet d'exécuter un moyennage du modèle de calcul avec des réseaux de neurones (*https://arxiv.org/abs/1207.0580*). Le terme "décrochage" se réfère à une suppression temporaire de neurones un réseau de neurones (*https://jmlr.org/papers/v15/srivastava14a.html*).

Ainsi, le réseau neuronal se voit amputé d'une partie de ses neurones pendant la phase d'entrainement (leur valeur est estimée à 0) et ils sont par contre réactivés pour tester le nouveau modèle.

In [ ]:
# Couche "entièrement connectée"
x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(rate=0.7)(x)
x = Dense(units=128, activation='relu')(x)
x = Dropout(rate=0.5)(x)
x = Dense(units=64, activation='relu')(x)
x = Dropout(rate=0.3)(x)

### Fonction sigmoïde

Jusqu'à présent, j'ai utilisé la fonction Relu. Pour la dernière couche du réseau, j'ai exceptionnellement appliqué la fonction "sigmoïde", tout simplement parce que, à la fin, notre problème est fondamentalement binaire (NORMAL ou PNEUMONIE). En effet, la fonction Sigmoïde donne une valeur entre 0 et 1, une probabilité. Elle est donc très utilisée pour les classification binaire lorsqu’un modèle doit déterminer seulement deux labels, ce qui est notre cas. f(x)= (1+e^(-x))^(-1)

In [ ]:
# Couche de sortie
output = Dense(units=1, activation='sigmoid')(x)

### L'optimiseur Adam

L'optimiseur Adam est une extension de la descente sctochastique de gradient et permet d'attribuer des poids aux valeurs des vecteurs. Le nom Adam vient de "adaptative moment estimation". Adam est efficace et ne nécessite qu'une faible mémoire de calcul, il est adapté au traitement des grandes quantités de données.

### La couche de perte (LOSS)

La couche de perte (LOSS) précise la différence entre le résultat attendu et le résultat obtenu. Ainsi, elle est généralement placée à la fin du réseau. Ici, on utilise la fonction d'entropie binaire croisée ("binary crossentropy") avec fonction sigmoïde en entrée d'activation. L'entropie binaire croisée est une fonction de perte qui permet de traiter des situations binaires, comme notre cas de diagnostic médical.

La perte par entropie binaire croisée couplée à une fonction sigmoïde est utilisée pour prédire K mesures de probabilité indépendante dans [0,1].

In [ ]:
 Création du modèle et compilation
model = Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Fonctions de rappel (callback)

Une fonction de rappel sert d’argument à toute autre fonction. Ces fonctions de rappel entrent généralement en jeu lorsque des fonctions asynchrones sont utilisées dans le programme. Une fonction synchrone s'exécute ligne après ligne en attendant la fin de l'exécution de la ligne précédente, alors on peut facilement en déduire que du code asynchrone va s'exécuter ligne après ligne, mais la ligne suivante n'attendra pas que la ligne asynchrone ait fini son exécution, ce qui permet des appels de fonctions de rappel.

- ModelCheckpoint : l'entraînement du réseau nécessite du temps pour obtenir un bon résultat. Pour cela, il est exécuté avec plusieurs epochs. Pendant l'exécution du programme, ModelCheckpoint sauvegarde, à la fin de chaque itération, la meilleure performance du modèle.

- EarlyStopping : c'est utile pour stopper le processus de généralisation lorsque la différence entre les résultats d'entraînement et les données de validation commence à trop grandir, en cas d'overfitting.

In [ ]:
# Fonctions de rappel
checkpoint = ModelCheckpoint(filepath='best_weights.hdf5', save_best_only=True, save_weights_only=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=1, mode='min')

### Entraînement du modèle

J'ai ensuite entraîné le modèle pendant 10 epochs avec un batch de taille 32. 
Nous pouvons remarquer qu'une plus grande taille de batch donne de meilleurs résultats mais consomme néanmoins énormément de ressources de calcul.

Des recherches ont montré qu'il y a une taille de batch optimale pour obtenir les meilleurs résultats sans trop consommer de ressources et que cette taille pourrait être obtenue en s'intéressant plus en profondeur aux "high-parameter tuning", mais cette étude pourrait constituer à elle-même un sujet de TIPE à part entière donc nous nous contenterons de nos paramètres.

In [ ]:
hist = model.fit_generator(
           train_gen, steps_per_epoch=train_gen.samples // batch_size, 
           epochs=epochs, validation_data=test_gen, 
           validation_steps=test_gen.samples // batch_size, callbacks=[checkpoint, lr_reduce])

### Evaluation du modèle
On veut désormais visualiser les métriques qui permettent d'évaluer les performances du modèle.

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

preds = model.predict(test_data)

accuracy = accuracy_score(test_labels, np.round(preds))*100
cm = confusion_matrix(test_labels, np.round(preds))
tn, fp, fn, tp = cm.ravel()

print('MATRICE DE CONFUSION ---------------------')
print(cm)

print('\nMETRIQUES DE TEST ----------------------')
precision = tp/(tp+fp)*100
recall = tp/(tp+fn)*100
print('Exactitude : {}%'.format(accuracy))
print('Précision : {}%'.format(precision))
print('Rappel : {}%'.format(recall))
print('Score F1 : {}'.format(2*precision*recall/(precision+recall)))

print('\nMETRIQUES D\'ENTRAINEMENT --------------')
print('Exactitude de l'entraînement : {}'.format(np.round((hist.history['accuracy'][-1])*100, 2)))
plt.show()

### Conclusion

Le modèle est capable d'atteindre une précision de 91.02% en 10 epochs, ce qui est très prometteur car, comparé aux entreprises et aux secteurs médicaux, j'ai eu accès à très peu de données d'entraînement. Des ordinateurs plus puissants permettraient également d'augmenter le nombre d'epochs et la taille des batchs, ce qui améliorerait grandement les résultats.